In [2]:
import os
import numpy as np
import pandas as pd

shuffle_data=True
dog_breed_train_path = 'Dogs/Ktop/all_train_5/'
dog_breed_val_path = 'Dogs/Ktop/all_val_5/'
dog_breed_labels_path = 'labels.csv'
labels = pd.read_csv(dog_breed_labels_path)

# to shuffle data
if shuffle_data:
    labels = labels.sample(frac=1)

In [3]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

n_classes = 5

Using TensorFlow backend.


In [4]:
from keras.applications.mobilenet import MobileNet

pic_dimension = 190
batch_size = 10
modelo_all_5 = MobileNet(input_shape=(pic_dimension, pic_dimension, 3), alpha=0.75, depth_multiplier=2, dropout=20e-2, include_top=True, weights=None, input_tensor=None, pooling='max', classes=n_classes)
modelo_all_5.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
modelo_all_5.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 190, 190, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 95, 95, 24)        648       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 95, 95, 24)        96        
_________________________________________________________________
conv1_relu (Activation)      (None, 95, 95, 24)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 95, 95, 48)        432       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 95, 95, 48)        192       
_________________________________________________________________
conv_dw_1_relu (Activation)  (None, 95, 95, 48)        0         
__________

In [6]:
from keras.preprocessing.image import ImageDataGenerator

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rotation_range=20,
        rescale=1./255,
        width_shift_range=0.3,
        height_shift_range=0.3,
        shear_range=0.3,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

# this is the augmentation configuration we will use for validation:
# only rescaling
val_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        dog_breed_train_path,  # this is the target directory
        target_size=(pic_dimension, pic_dimension),  # all images will be resized to 112x112
        batch_size=batch_size,
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = val_datagen.flow_from_directory(
        dog_breed_val_path,
        target_size=(pic_dimension, pic_dimension),
        batch_size=batch_size,
        class_mode='categorical')

results = modelo_all_5.fit_generator(
        train_generator, 
        steps_per_epoch=607//batch_size, # tamaño del training set//tamaño del batch 
        epochs=100,
        validation_data=validation_generator,
        validation_steps=154//batch_size,
        #callbacks=[lrate]
        )
modelo_all_5.save_weights('modelo_all_5.h5')

Found 607 images belonging to 5 classes.
Found 154 images belonging to 5 classes.
Epoch 1/100
60/60 [==============================] - 288s 5s/step - loss: 1.5696 - acc: 0.3191 - val_loss: 5.5199 - val_acc: 0.2067
Epoch 2/100
60/60 [==============================] - 288s 5s/step - loss: 1.6095 - acc: 0.3195 - val_loss: 2.2176 - val_acc: 0.2431
Epoch 3/100
60/60 [==============================] - 288s 5s/step - loss: 1.5244 - acc: 0.3488 - val_loss: 2.2974 - val_acc: 0.3542
Epoch 4/100
60/60 [==============================] - 292s 5s/step - loss: 1.5154 - acc: 0.3467 - val_loss: 2.2006 - val_acc: 0.2222
Epoch 5/100
60/60 [==============================] - 288s 5s/step - loss: 1.4533 - acc: 0.3700 - val_loss: 4.9177 - val_acc: 0.2153
Epoch 6/100
60/60 [==============================] - 303s 5s/step - loss: 1.4008 - acc: 0.4378 - val_loss: 5.7102 - val_acc: 0.2153
Epoch 7/100
60/60 [==============================] - 303s 5s/step - loss: 1.3781 - acc: 0.4362 - val_loss: 3.8244 - val_acc: 0

KeyboardInterrupt: 